# The quandle counting polynomial invariant of knotted 4-valent graphs

This notebook is intended to show the process of finding the quandle counting polynomial invariant $\Psi_Q (G)$ for a choice of knotted 4-valent graph $G$, and a quandle $Q$. It uses Python code from the package Graphpoly to do this; here are the steps used, given a specific Dowker-Thistletwaite sequence for a knotted graph.

1. Find the orientation of the graph nodes by running the DT algorithm on the sequence, giving the $f(i)$ function on node labels $i$. The convention is that $f(i) = +1$ if the other arc crosses from right to left, and $f(i) = -1$ if the other arc crosses from left to right.
1. Find the planar diagram (PD) code for the graph, using the DT sequence and the orientation just found.
1. From the PD code, find all possible Eulerian circuits through the graph. This gives the multiset of induced knot diagrams for the knot.
1. For each induced knot diagram, find the number of quandle colorings of the knot, with the chosen quandle $Q$.
1. Construct the polynomial invariant $\Psi_Q (G)$ by writing the generating function for the multiset of quandle colorings.

For this notebook, we will use the example of the graph given by the DT sequence $3^\ell 5^u 7^- 1^+$. This is represented as a Python list by

```python
    seq = [[0, 3, -2], [2, 5, 2], [4, 7, -1], [6, 1, 1]]
```

For each node, there is a list of three numbers in this list. The first element is an even number, the second an odd; these are the two labels for that particular node. The node labels are in the set $\{0, 1, \cdots, 2N - 1\}$, for a graph with $N$ nodes. The third element gives the crossing or vertex type: -2 for a vertex with the upper edge on the odd label (label superscript $\ell$), -1 for a crossing with the upper edge on the odd label (label superscript $-$), and the positive numbers are the flipped cases (superscripts $u$ and $+$).

## Find the orientation of the graph nodes

First, we import the function `isRealizable(seq, f_list = False)`. If the function is given a Python list `seq`, the function returns `True` if the sequence is a realizable DT sequence, and `False` if the sequence does not represent a planar graph. Note that the given sequence `seq` may or may not include the crossing or vertex type. If the parameter `f_list` is set to `True`, for a realizable sequence, the function will return a list of $f(i)$ values for each node label $i$. These indicate the crossing information at the node $i$ for the Eulerian circuit described by the DT sequence. See "Classification of knot projections", Topology Appl. 16, 19–31 (1983), for details.

In [1]:
from Graphpoly.isRealizable import isRealizable

The next command creates the orientation list `orientList` for the example DT sequence `seq`. Remember that DT sequences do not differentiate between a sequence and its mirror image. Thus, one can make a global change in sign of the $f(i)$ for all $i$, and still have a realizable sequence.

In [2]:
# Example sequence 3^l 5^u 7^- 1^+

seq = [[0, 3, -2], [2, 5, 2], [4, 7, -1], [6, 1, 1]]

orientList = isRealizable(seq, f_list = True)
orientList

[1, 1, -1, -1, 1, 1, -1, -1]

## Find the planar diagram code

The function `planarDiagram(seq, f_list)` takes the DT sequence `seq` and the orientation list `f_list`, and produces a planar diagram list from them. Thus, the function returns a list of two lists. The first list gives the PD code for the graph. **Note:** the numbers in this list refer to *half-edges* (or *darts*), not *nodes*. There can be problems if an induced knot diagram has self-loops, so using half-edges allows for a unique number associated with each of the four half-edges incident to the node.

A vertex is treated as the appropriate crossing, based on its vertex state. The second list gives the node type for each node. Again, we use 1 for a crossing, and 2 for a vertex. The sign of the numbers comes from $f(i)$, so they are positive if the upper crossing is right-to-left, and negative if left-to-right.

In [3]:
from Graphpoly.planarDiagram import planarDiagram

Here, we show the planar diagram notation for $3^\ell 5^u 7^- 1^+$, where the orientation is that found above in a previous cell. The node type, along with the value of $f(i)$, is recorded in `nodeTypeList`.

In [4]:
[PD_code, nodeTypeList] = planarDiagram(seq, orientList)
PD_code

[[0, 6, 1, 7], [2, 12, 3, 13], [8, 14, 9, 15], [10, 4, 11, 5]]

## List all induced knot diagrams

We use a modified version of Hierholzer's algorithm to find all possible Eulerian circuits through the graph. Note that there is only one possible way to pass along the circuit when coming into a crossing, but three possible ways for a vertex. Thus, the algorithm must take this freedom into account, using the list `nodeTypeList` obtained from `planarDiagram()`.

In [5]:
from Graphpoly.createCircuits import createCircuits

The function `createCircuits()` returns a list of `graph` objects, each of which represents a particular choice of Eulerian circuit through the original graph. The half-edges of the circuit can be returned using `graph.listEPath()`. The labels of the half-edges are the same as those used in `planarDiagram()`.

In [6]:
Ecircuits = createCircuits(PD_code, nodeTypeList)
graphList = [graph for graph in Ecircuits]

for graph in graphList:
    print(graph.listEPath())
    
print('Number of circuits:', len(graphList))

[0, 15, 14, 13, 12, 11, 4, 3, 2, 1, 6, 5, 10, 9, 8, 7]
[0, 7, 8, 9, 10, 5, 6, 1, 2, 3, 4, 11, 12, 13, 14, 15]
[0, 6, 5, 10, 9, 8, 7, 1, 2, 3, 4, 11, 12, 13, 14, 15]
[0, 15, 14, 13, 12, 11, 4, 3, 2, 1, 7, 8, 9, 10, 5, 6]
[0, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
[0, 15, 14, 13, 12, 11, 10, 9, 8, 7, 1, 2, 3, 4, 5, 6]
[0, 6, 5, 4, 3, 2, 1, 7, 8, 9, 10, 11, 12, 13, 14, 15]
[0, 15, 14, 13, 12, 11, 5, 6, 1, 2, 3, 4, 10, 9, 8, 7]
[0, 7, 8, 9, 10, 4, 3, 2, 1, 6, 5, 11, 12, 13, 14, 15]
[0, 1, 2, 3, 4, 10, 9, 8, 7, 6, 5, 11, 12, 13, 14, 15]
[0, 15, 14, 13, 12, 11, 5, 6, 7, 8, 9, 10, 4, 3, 2, 1]
Number of circuits: 12


## PD codes for induced knot diagrams

Now that we have lists of half-edges that give all of the induced knot diagrams for a given graph, we write these in terms of planar diagram codes. This is done by the function `createInducedKnot()`.

In [7]:
from Graphpoly.createInducedKnot import createInducedKnot

The function `createInducedKnot(circuit, PD_code)` takes a list `circuit` produced by `createCircuits()`, along with the PD code `PD_code` for the original knotted graph. It returns the PD code for the induced knot diagram, where the labels for the half-edges have been renumbered, to ensure that for an induced knot with $N$ crossings, all half-edge labels are in the set $\{0, 1, ..., 4N\}$.

In [8]:
for graph in graphList:
    print(graph.listEPath(), '->', createInducedKnot(graph))

[0, 15, 14, 13, 12, 11, 4, 3, 2, 1, 6, 5, 10, 9, 8, 7] -> [[[1, 0, 0, 0], [0, 0, 1, 0]], [1, 1]]
[0, 7, 8, 9, 10, 5, 6, 1, 2, 3, 4, 11, 12, 13, 14, 15] -> [[[0, 0, 1, 0], [1, 0, 0, 0]], [1, 1]]
[0, 6, 5, 10, 9, 8, 7, 1, 2, 3, 4, 11, 12, 13, 14, 15] -> [[[0, 0, 1, 0], [1, 0, 0, 0]], [-1, 1]]
[0, 15, 14, 13, 12, 11, 4, 3, 2, 1, 7, 8, 9, 10, 5, 6] -> [[[1, 0, 0, 0], [0, 0, 1, 0]], [-1, 1]]
[0, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1] -> [[[3, 0, 2, 0], [2, 0, 1, 0], [0, 2, 3, 2], [1, 2, 0, 2]], [1, 1, 1, 1]]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] -> [[[0, 2, 1, 2], [1, 0, 2, 0], [3, 2, 0, 2], [2, 0, 3, 0]], [1, 1, 1, 1]]
[0, 15, 14, 13, 12, 11, 10, 9, 8, 7, 1, 2, 3, 4, 5, 6] -> [[[2, 0, 1, 0], [1, 0, 0, 0], [0, 0, 2, 0]], [1, -1, -1]]
[0, 6, 5, 4, 3, 2, 1, 7, 8, 9, 10, 11, 12, 13, 14, 15] -> [[[2, 0, 0, 0], [0, 0, 1, 0], [1, 0, 2, 0]], [-1, -1, 1]]
[0, 15, 14, 13, 12, 11, 5, 6, 1, 2, 3, 4, 10, 9, 8, 7] -> [[[1, 0, 0, 0], [0, 0, 1, 0]], [1, -1]]
[0, 7, 8, 9, 10, 4

## Linear Alexander quandles

Now that we have a list of induced knot diagrams for the original knotted graph, we need a way of finding the quandle counting polynomial for these diagrams. This can be done with linear algebra when we use a class of quandles known as *linear Alexander quandles*. For a number $n$, and a number $t$ relatively prime to $n$, the Alexander quandle $\Lambda_{n, t}$ is defined on integers $x, y \in \mathbb{Z}_n$ by
$$
    x \triangleleft y = t x + (1 - t) y
$$
The inverse $\triangleleft^{-1}$ of this operation is given by
$$
    x \triangleleft^{-1} y = t^{-1} x + (1 - t^{-1}) y
$$
where $t^{-1}$ is the inverse of $t$ mod $n$. Since $n$ and $t$ are relatively prime, there is a unique inverse.

Suppose there is a crossing in the induced knot diagram, with the edges colored by $x_i, x_j, x_k \in \mathbb{Z}_n$. Then, testing whether these colors are a valid coloring of the knot with $x_i \triangleleft x_j = x_k$ by quandle elements is equivalent to solving the equation
$$
    t x_i + (1 - t)x_j - x_k = 0
$$
Thus, finding all allowed quandle colorings of an $N$ crossing knot is the same as finding the nullspace of the $N \times N$ matrix, whose entries are determined by the equation above.

To help in this process, we first define a function `extEuclid(n, t)` which uses the extended Euclidean algorithm to find the greatest common divisor (GCD) of two integers $n, t$.

In [9]:
"""
This program uses the Euclidean algorithm to find the inverse of t mod n. This
is done by solving the equation

    an + bt = 1
    
for the number b. The method uses recursive calls to the algorithm, as outlined
on the page https://theprogrammingexpert.com/euclidean-algorithm-python/
"""

def extEuclid(n, t):
    
    if n == 0:
        return t, 0, 1
    
    gcd, u, v = extEuclid(t % n, n)
    x = v - (t // n) * u
    y = u
    
    return gcd, x, y

Then, if the GCD of $n, t$ is 1, we can define a linear Alexander quandle $\Lambda_{n, t}$.

In [10]:
import sympy as sp

In [11]:
# Define numbers n, t

N = 3      # Must be prime
t = 2

(gcd, a, b) = extEuclid(N, t)

if gcd != 1:
    print('N, t not relatively prime')
else:
    Q = sp.Matrix([[(t * iii + (1 - t) * jjj) % N for jjj in range(N)] for iii in range(N)])
    display(Q)

Matrix([
[0, 2, 1],
[2, 1, 0],
[1, 0, 2]])

Now we use this operation matrix to find the appropriate matrix to solve for the number of quandle colorings for a given induced knot.

In [12]:
from sympy import Matrix, Poly, GF
from sympy.matrices.normalforms import smith_normal_form
from sympy.abc import u

In [13]:
# Create dictionary to hold multiset information

coeffDict = {}

# Go through each induced knot diagram for the graph;
# find number of quandle colorings for the induced knot diagram

for kkk in range(len(graphList)):
    
    # Definitions
    
    [knot, f_list] = createInducedKnot(graphList[kkk])
    circuit = graphList[kkk].listEPath()

    num_nodes = len(knot)
    
    # Special cases
    
    if knot == [] or knot == [[0, 0, 0, 0]]:
        coeffDict[(N, )] = coeffDict.get((N, ), 0) + 1
        continue
        
    # General case with knot having more than one crossing
    
    M = Matrix([[0 for jjj in range(num_nodes)] for iii in range(num_nodes)])
    
    # Go through each node, and add appropriate linear equation coefficients to M
    
    for iii in range(num_nodes):
        node, f = knot[iii], f_list[iii]

        if node[0] == node[1]:          # Self-loop; entering edge is undercrossing
            M[iii, node[0]] = 1
            M[iii, node[2]] = N - 1
            continue

        if node[1] == node[2]:          # Self-loop; entering edge is overcrossing
            M[iii, node[0]] = 1
            M[iii, node[2]] = N - 1
            continue

        if f == 1:                      # Overcrossing is right-to-left
            M[iii, node[0]] = N - 1
            M[iii, node[1]] = (1 - t) % N
            M[iii, node[2]] = t

        if f == -1:                     # Overcrossing is left-to-right
            M[iii, node[0]] = t
            M[iii, node[1]] = (1 - t) % N
            M[iii, node[2]] = N - 1
            
    # Find null space of matrix M
            
    A = smith_normal_form(M, domain = GF(N))
    dim = len(A.nullspace())
    
    coeffDict[(N ** dim, )] = coeffDict.get((N ** dim, ), 0) + 1
    
# Create generating function for multiset

f = Poly.from_dict(coeffDict, u)
print('n, t, f =', N, t, f)

n, t, f = 3 2 Poly(2*u**9 + 10*u**3, u, domain='ZZ')


## Directing counting

The matrix method above works only for a linear Alexander quandle. For an arbitrary quandle, we can directly count the number of quandle colorings of the graph. The quandle can be defined by a matrix $M_{ij}$, giving the results $x_i \triangleleft x_j$ of the quandle operation. An example is shown below.

In [14]:
Q = sp.Matrix([[0, 0, 0, 0], [1, 1, 3, 2], [2, 3, 2, 1], [3, 2, 1, 3]])
display(Q)

Matrix([
[0, 0, 0, 0],
[1, 1, 3, 2],
[2, 3, 2, 1],
[3, 2, 1, 3]])

The code below goes through a similar algorithm to that given above, going through each induced knot diagram for the original knotted graph. However, instead of constructing a matrix using a linear Alexander quandle, the number of quandle colorings is counted by trying each possible crossing.

In [15]:
from itertools import product

In [16]:
# Create dictionary to hold multiset information

coeffDict = {}

# Number of elements in quandle

N = sp.shape(Q)[0]

# Go through each induced knot diagram for the graph;
# find number of quandle colorings for the induced knot diagram

for kkk in range(len(graphList)):
    
    # Definitions
    
    [knot, f_list] = createInducedKnot(graphList[kkk])
    circuit = graphList[kkk].listEPath()

    num_nodes = len(knot)
    num_colorings = 0
    
    # Special cases
    
    if knot == [] or knot == [[0, 0, 0, 0]]:
        coeffDict[(N, )] = coeffDict.get((N, ), 0) + 1
        continue
    
    # General case with knot having more than one crossing; there are
    # num_nodes total edges, since every edge on an oriented knot (not a 
    # link!) starts at a crossing.

    for colorList in product(range(N), repeat = num_nodes):
        colorList = list(colorList)

        flag = False
        iii = 0

        while iii < (num_nodes - 1):

            node, f = knot[iii], f_list[iii]

            if node[0] == node[1] or node[1] == node[2]:    # Self-loop
                if colorList[node[0]] != colorList[node[1]] != colorList[node[2]] != colorList[node[3]]:
                    flag = True
                    break

            if f == 1:                                      # Overcrossing is right-to-left
                if Q[colorList[node[2]], colorList[node[1]]] != colorList[node[0]] \
                    or colorList[node[1]] != colorList[node[3]]:
                    flag = True
                    break

            if f == -1:                                     # Overcrossing is left-to-right
                if Q[colorList[node[0]], colorList[node[1]]] != colorList[node[2]] \
                    or colorList[node[1]] != colorList[node[3]]:
                    flag = True
                    break

            iii += 1

        if not flag:            # All nodes pass required tests
            num_colorings += 1

    coeffDict[(num_colorings, )] = coeffDict.get((num_colorings, ), 0) + 1

# Print polynomial obtained

f = Poly.from_dict(coeffDict, u) 
print(f)

Poly(2*u**10 + 10*u**4, u, domain='ZZ')
